In [1]:
class SshConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

class MysqlConfig {
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

In [2]:
class SshTunneledDb2Connection 
{
    public $ssh_config;
    public $mysql_config;
    
    public function __construct(SshConfig $ssh_config, MysqlConfig $mysql_config)
    {
        $this->ssh_config = $ssh_config;
        $this->mysql_config = $mysql_config;
    }
    
    public function query(string $query)
    {
        
        //exec('ssh -f -L 3307:127.0.0.1:3306 user@example.com sleep 10 > /dev/null');
        
        $connection = ssh2_connect($this->ssh_config->hostname, $this->ssh_config->port);
        if($connection === FALSE){
            throw new RuntimeException("Failed to connect with SSH server.");
        }

        if(!ssh2_auth_password($connection, $this->ssh_config->username, $this->ssh_config->password)){
              throw new RuntimeException("Failed to authenticate to SSH server");
        }
        
        $tunnel = ssh2_tunnel($connection, $this->mysql_config->hostname, 3306);
        
        while (!feof($tunnel)) 
        {
            mysqli_report(MYSQLI_REPORT_ERROR | MYSQLI_REPORT_STRICT);
            $mysqli = new mysqli(
                $this->mysql_config->hostname, 
                $this->mysql_config->username, 
                $this->mysql_config->password, 
                'nanohub',
                3306
            );
            if ($db->connect_error) {
                    die('Connect Error (' . $mysqli->connect_errno . ') '
                        . $mysqli->connect_error);
            } elseif ($result = $mysqli->query("SELECT DATABASE()")) {
                    $row = $result->fetch_row();
                    if ($row[0] != 'nanohub') {
                            //oops! We're connected to mysql, but not to database_b
                    }
            }
        }
    }
}

class SshTunneledConnectionFactory
{
    static public function create_from_environment()
    {
        $ssh_config = new SshConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $mysql_config = new MysqlConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return new SshTunneledDb2Connection($ssh_config, $mysql_config);
    }
}

$connection = SshTunneledConnectionFactory::create_from_environment();
print_r($connection->query("SELECT * FROM nanohub.jos_faq"));

mysqli_sql_exception with message 'Connection refused'

In [3]:
// $ssh['username'] = getenv('tunnel_ssh_username');
// $ssh['password'] = getenv('tunnel_ssh_password');
// $ssh['hostname'] = getenv('tunnel_ssh_host');
// $ssh['port'] = getenv('tunnel_ssh_port');

// $mysql['username'] = getenv('db_user');
// $mysql['password'] = getenv('db_password');
// $mysql['hostname'] = getenv('db_host');
// $mysql['port'] = getenv('tunnel_remote_bind_port');

In [4]:
$query = 'SELECT * FROM nanohub.jos_faq';

$ssh_query ='ssh -L 3307:' . getenv('tunnel_ssh_host').':' . getenv('tunnel_remote_bind_port').'; 
echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.getenv('db_user').' -h '.getenv('db_host').' --password='.getenv('db_password');

$response = ssh2_exec($connection, $ssh_query);
$error_response = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
print_r(stream_get_contents($error_response));

stream_set_blocking($response, true);
stream_set_blocking($error_response, true);

$results = stream_get_contents($response);
if (is_null($results ))
    echo "Empty response/result";

else 
    print_r($results);

ssh2_disconnect ( $connection );

PHP Warning:  ssh2_exec() expects parameter 1 to be resource, object given in /home/saxenap/.jupyter-php/pkgs/vendor/litipk/jupyter-php/src/Actions/ExecuteAction.php(115) : eval()'d code on line 4